- - -
**File name**: horizontal_wrf_roms.ipynb

**Author**: Ueslei Adriano Sutil

**Email**: [ueslei@outlook.com](mailto:ueslei@outlook.com)

**Created**: 09 October 2020

**Last modified**: 26 October 2020

**Version**: 1.6

**Python**: 3.7.9
- - -

Create horizontal plots from ROMS (his) and WRF-ARW outputs, displaying:
- Sea surface temperature (Contourf; °C);
- Sea surface salinity (Contourf; PSU);
- Heat fluxes (Contourf; W m-2);
- 200 and 1000 meters bathymetry (Contour; m);
- Wind vectors at 10 m (Vector; m s-1);
- Ocean current at surface (Vector; m s-1);
- Sea level pressure (Contourf; hPa);
- Daily Precipitation (Contourf; mm);
- Convective Avaliable Potential Energy (CAPE; Contourf; J kg-1).
- Outgoing Longwave Radiation (Contourf; W m-2)

#### Warnings:
- This script convert ROMS SST from Potential Temperature to Temperature using the following equation from MetPy package [[Access here]](https://unidata.github.io/MetPy/latest/installguide.html):
$
\begin{equation}
Temp = \Theta(\frac{P}{P_{0}})^k 
\end{equation}
$

- bbox = lon_min,lon_max,lat_min,lat_max

#### 1. Import libraries.

In [1]:
from   tqdm                  import tqdm
from   tqdm.notebook         import trange, tqdm
from   time                  import sleep
from   wrf                   import to_np, getvar, latlon_coords, smooth2d, extract_times, cape_2d
from   scipy.ndimage.filters import gaussian_filter
from   ipywidgets            import VBox,ToggleButtons, IntSlider, Tab, SelectionRangeSlider, IntProgress
from   netCDF4               import Dataset
from   matplotlib            import path 
from   mpl_toolkits.basemap  import Basemap
import pandas                as     pd
import numpy                 as     np
import matplotlib.pyplot     as     plt
import matplotlib.colors     as     colors
import os
import cmocean

#### 2. Create some definitions that will be later used in the script.

In [2]:
def create_bathy(lon_1=-49,lon_2=-33,lat_1=-34,lat_2=-20):
    """Plot ETOPO1 data as a bathymetry contour.
    """
    # Open data.
    etopodata = Dataset('/media/ueslei/Ueslei_HD/Scripts/Python/Auxiliares/OceanLab/ETOPO1.nc')

    # Read data.
    etopo_lon = etopodata.variables['x'][:]
    etopo_lat = etopodata.variables['y'][:]

    # Create subset.
    x_data = (etopo_lon>lon_1)&(etopo_lon<lon_2)
    y_data = (etopo_lat>lat_1)&(etopo_lat<lat_2)
    bathy_topo = etopodata.variables['z'][y_data,:][:,x_data]

    # Make grid.
    bathy_lon,bathy_lat = np.meshgrid(etopo_lon[x_data],etopo_lat[y_data])

    return bathy_lon,bathy_lat,bathy_topo

def bbox2ij(lon,lat,bbox=[-160., -155., 18., 23.]):
    """Return indices for i,j that will completely cover the specified bounding box.

    i0,i1,j0,j1 = bbox2ij(lon,lat,bbox)

    lon,lat = 2D arrays that are the target of the subset
    bbox = list containing the bounding box: [lon_min, lon_max, lat_min, lat_max]

    Example
    -------  
    i0,i1,j0,j1 = bbox2ij(lon_rho,[-71, -63., 39., 46])
    h_subset = nc.variables['h'][j0:j1,i0:i1]       
    """
    bbox=np.array(bbox)
    mypath=np.array([bbox[[0,1,1,0]],bbox[[2,2,3,3]]]).T
    p = path.Path(mypath)
    points = np.vstack((lon.flatten(),lat.flatten())).T
    n,m = np.shape(lon)
    inside = p.contains_points(points).reshape((n,m))
    ii,jj = np.meshgrid(range(m),range(n))
    return min(ii[inside]),max(ii[inside]),min(jj[inside]),max(jj[inside])

def rot2d(x, y, ang):
    """Rotate vectors by geometric angle
        
    This routine is part of Rob Hetland's OCTANT package:
    https://github.com/hetland/octant
    """
    xr = x*np.cos(ang) - y*np.sin(ang)
    yr = x*np.sin(ang) + y*np.cos(ang)
    return xr, yr

def shrink(a,b):
    """Return array shrunk to fit a specified shape by triming or averaging.
    
    a = shrink(array, shape)
    
    array is an numpy ndarray, and shape is a tuple (e.g., from
    array.shape). a is the input array shrunk such that its maximum
    dimensions are given by shape. If shape has more dimensions than
    array, the last dimensions of shape are fit.
    
    as, bs = shrink(a, b)
    
    If the second argument is also an array, both a and b are shrunk to
    the dimensions of each other. The input arrays must have the same
    number of dimensions, and the resulting arrays will have the same
    shape.
    
    This routine is part of Rob Hetland's OCTANT package:
    https://github.com/hetland/octant
    
    Example
    -------
    
    >>> shrink(rand(10, 10), (5, 9, 18)).shape
    (9, 10)
    >>> map(shape, shrink(rand(10, 10, 10), rand(5, 9, 18)))        
    [(5, 9, 10), (5, 9, 10)]   
    """

    if isinstance(b, np.ndarray):
        a = shrink(a,b.shape)
        b = shrink(b,a.shape)
        return (a, b)

    if isinstance(b, int):
        b = (b,)

    if len(a.shape) == 1:                # 1D array is a special case
        dim = b[-1]
        while a.shape[0] > dim:          # only shrink a
            if (dim - a.shape[0]) >= 2:  # trim off edges evenly
                a = a[1:-1]
            else:                        # or average adjacent cells
                a = 0.5*(a[1:] + a[:-1])
    else:
        for dim_idx in range(-(len(a.shape)),0):
            dim = b[dim_idx]
            a = a.swapaxes(0,dim_idx)        # put working dim first
            while a.shape[0] > dim:          # only shrink a
                if (a.shape[0] - dim) >= 2:  # trim off edges evenly
                    a = a[1:-1,:]
                if (a.shape[0] - dim) == 1:  # or average adjacent cells
                    a = 0.5*(a[1:,:] + a[:-1,:])
            a = a.swapaxes(0,dim_idx)        # swap working dim back

    return a

class MidpointNormalize(colors.Normalize):
	"""
	Normalise the colorbar so that diverging bars work there way either side from a prescribed midpoint value)

	e.g. im=ax1.imshow(array, norm=MidpointNormalize(midpoint=0.,vmin=-100, vmax=100))
	"""
	def __init__(self, vmin=None, vmax=None, midpoint=None, clip=False):
		self.midpoint = midpoint
		colors.Normalize.__init__(self, vmin, vmax, clip)

	def __call__(self, value, clip=None):
		# I'm ignoring masked values and all kinds of edge cases to make a
		# simple example...
		x, y = [self.vmin, self.midpoint, self.vmax], [0, 0.5, 1]
		return np.ma.masked_array(np.interp(value, x, y), np.isnan(value))

#### 2. Customize script options.

In [7]:
plot_bathy = ToggleButtons(
    options=[("Yes", True), ("No", False)],
    description='Bathymetry?',
    button_style='info',
)
plot_bathy.style.button_width = '100px'

plot_currents = ToggleButtons(
    options=[("Yes", True), ("No", False)],
    description='Currents?',
    button_style='info',
)
plot_currents.style.button_width = '100px'

plot_wind = ToggleButtons(
    options=[("Yes", True), ("No", False)],
    description='Wind?',
    button_style='info',
)
plot_wind.style.button_width = '100px'

plot_slp = ToggleButtons(
    options=[("Yes", True), ("No", False)],
    description='SLP?',
    button_style='info',
)
plot_slp.style.button_width = '100px'

create_video = ToggleButtons(
    options=[("Yes", True), ("No", False)],
    description='Video?',
    button_style='info',
)
create_video.style.button_width = '100px'

ppt_fig = ToggleButtons(
    options=[("Yes", True), ("No", False)],
    description='Blank BG?',
    button_style='info',
)
ppt_fig.style.button_width = '100px'

box = VBox([plot_bathy,plot_currents,plot_wind,plot_slp,create_video,ppt_fig])
display(box)

#### 3. Select which variable will be plotted (as countour).

In [9]:
variable = ToggleButtons(
    options=[("SST", 1), ("Salinity", 2),("Daily Prec.", 3.1),("Accum. Prec.", 3.2),("Heat Fluxes", 4),("CAPE", 5),("OLR",6)],
    description='Variable:',
    button_style='info',
)
variable.style.button_width = '100px'
display(variable)

ToggleButtons(button_style='info', description='Variable:', options=(('SST', 1), ('Salinity', 2), ('Daily Prec…

#### 4. Load files and variables.

In [10]:
# Open files.
roms_file = '/media/ueslei/Ueslei_HD/INPE/PCI/Projetos/SC_2008/Outputs/normal/roms.nc'
wrf_file  = '/media/ueslei/Ueslei_HD/INPE/PCI/Projetos/SC_2008/Outputs/normal/wrf.nc'
nc_roms   = Dataset(roms_file)
nc_wrf    = Dataset(wrf_file)

# Set map limits and ROMS level.
bbox = [-53,-44,-31,-23]
zlev = -1 # Last sigma layer corresponds to surface.

# Count how many time-steps.
ntimes    = len(nc_wrf.variables['Times'][:])
loop      = [i for i in range(97,ntimes,1)]
min_loop  = min(loop)
max_loop  = max(loop)
len_loop  = len(loop)

# Load dates.
timestr1 = extract_times(nc_wrf,timeidx=None,meta=False,do_xtime=False)
timestr  = pd.to_datetime(timestr1, format="%b %d %Y %H:%M")
timestr  = timestr[97::]

# Load WRF/ROMS lat-lon coords as a numpy array.
# NOTE: do not change the ifs below to elif.
if plot_wind.value == True or plot_slp.value == True or variable.value == 3 or variable.value == 4:
    uvmet10 = getvar(nc_wrf, "uvmet10", 0, units="m s-1")
    lats_wrf, lons_wrf = latlon_coords(uvmet10,as_np=True)    
    lats_wrf2 = lats_wrf[:,:] 
    lons_wrf2 = lons_wrf[:,:]
    lats_wrf  = lats_wrf[:,0]
    lons_wrf  = lons_wrf[0,:]

if variable.value == 1 or variable.value == 2 or variable.value == 3 or plot_wind.value == True:
    lon_rho     = nc_roms.variables['lon_rho'][:]
    lat_rho     = nc_roms.variables['lat_rho'][:]
    i0,i1,j0,j1 = bbox2ij(lon_rho,lat_rho,bbox)
    lon_var     = lon_rho[j0:j1, i0:i1]
    lat_var     = lat_rho[j0:j1, i0:i1]
    lon         = lon_rho[(j0+1):(j1-1), (i0+1):(i1-1)]
    lat         = lat_rho[(j0+1):(j1-1), (i0+1):(i1-1)]
if variable.value == 1 or variable.value == 2:
    var_roms    = np.zeros([len_loop,len(lat_var[:,0]),len(lon_var[0,:])]) 
if variable.value == 3.1 or variable.value == 3.2:
    var_wrf_1   = np.zeros([len_loop,len(lats_wrf),len(lons_wrf)])
    var_wrf_2   = np.zeros([len_loop,len(lats_wrf),len(lons_wrf)])   
    var_wrf_11  = np.zeros([len_loop,len(lats_wrf),len(lons_wrf)])
    var_wrf_22  = np.zeros([len_loop,len(lats_wrf),len(lons_wrf)])  
    var_wrf_111 = np.zeros([len_loop,len(lats_wrf),len(lons_wrf)])   
    var_wrf     = np.zeros([len_loop,len(lats_wrf),len(lons_wrf)])             
if variable.value == 4:
    var_wrf_1 = np.zeros([len_loop,len(lats_wrf),len(lons_wrf)])
    var_wrf_2 = np.zeros([len_loop,len(lats_wrf),len(lons_wrf)])   
if variable.value == 5:
    var_wrf = np.zeros([4,len_loop,len(lats_wrf),len(lons_wrf)])  
if variable.value == 6:
    var_wrf = np.zeros([len_loop,len(lats_wrf),len(lons_wrf)])

# Create variables to store data.
if plot_wind.value==True:
    # Since uvmet10 funcion store U and V variables inside 0 and 1 dimensions,
    # the first uvmet10 dimension is 2.
    uvmet10 = np.zeros([2,len_loop,len(lats_wrf),len(lons_wrf)])
    sleep(0.01) 
if plot_slp.value==True:
    slp = np.zeros([len_loop,len(lats_wrf),len(lons_wrf)])
    sleep(0.01) 
    caching.clear_cache()
if plot_currents.value==True:    
    u = np.zeros([len_loop,len(lat[:,0]),len(lon[0,:])])
    v = np.zeros([len_loop,len(lat[:,0]),len(lon[0,:])]) 
# Start looping through variables.
if plot_wind.value==True:
    for i in tqdm(range(len_loop), desc='Wind'):
        uvmet10[:,i,:,:] = getvar(nc_wrf, "uvmet10", loop[i], units="m s-1")  
        sleep(0.01) 
if plot_slp.value==True:
    for i in tqdm(range(len_loop), desc='SLP'):
        slp1       = getvar(nc_wrf, "slp", loop[i], units="hPa",)
        slp[i,:,:] = smooth2d(slp1,120)
        sleep(0.01)
if plot_currents.value==True:
    for i in tqdm(range(len_loop), desc='Currents'):
        lon_rho     = nc_roms.variables['lon_rho'][:]
        lat_rho     = nc_roms.variables['lat_rho'][:]
        i0,i1,j0,j1 = bbox2ij(lon_rho,lat_rho,bbox)
        lon_var     = lon_rho[j0:j1, i0:i1]
        lat_var     = lat_rho[j0:j1, i0:i1]
        lon         = lon_rho[(j0+1):(j1-1), (i0+1):(i1-1)]
        lat         = lat_rho[(j0+1):(j1-1), (i0+1):(i1-1)]
        u1          = nc_roms.variables['u'][loop[i], zlev, j0:j1, i0:(i1-1)]
        v1          = nc_roms.variables['v'][loop[i], zlev, j0:(j1-1), i0:i1]
        mask        = 1 - nc_roms.variables['mask_rho'][(j0+1):(j1-1), (i0+1):(i1-1)]
        ang         = nc_roms.variables['angle'][(j0+1):(j1-1), (i0+1):(i1-1)] 
        u1          = shrink(u1, mask.shape) #Average U to rho points.
        v1          = shrink(v1, mask.shape) # Average V points to rho points.
        u1, v1      = rot2d(u1, v1, ang) # Rotate grid oriented U and V to East and West.
        v1[v1.mask] = np.nan
        u1[u1.mask] = np.nan
        u[i,:,:]    = u1
        v[i,:,:]    = v1
        sleep(0.01)
if variable.value == 1:
    for i in tqdm(range(len_loop), desc='SST'):
        var_roms[i,:,:] = nc_roms.variables['temp'][loop[i], zlev,  j0:j1, i0:i1]
        var_roms[var_roms > 30000] = np.nan
        sleep(0.01) 
if variable.value == 2:
    for i in tqdm(range(len_loop), desc='Salt'):
        var_roms[i,:,:] = nc_roms.variables['salt'][loop[i], zlev,  j0:j1, i0:i1]
        var_roms[var_roms > 30000] = np.nan
        sleep(0.01) 
if variable.value == 3.1:
    for i in tqdm(range(len_loop), desc='Prec'):
        if i == 0:
            var_wrf_1[0,:,:] = nc_wrf.variables['RAINNC'][0,:,:]
            var_wrf[0,:,:]   = var_wrf_1[0,:,:]
            sleep(0.01) 
        elif i == 1:
            var_wrf_1[0,:,:] = nc_wrf.variables['RAINNC'][loop[1],:,:]
            var_wrf_2[0,:,:] = nc_wrf.variables['RAINC'][loop[1],:,:]  
            var_wrf[1,:,:]   = (var_wrf_2[0,:,:]+var_wrf_1[0,:,:])-(var_wrf[0,:,:]+var_wrf[0,:,:])
        elif i >= 2:
            var_wrf_1[i,:,:]  = nc_wrf.variables['RAINNC'][loop[i],:,:]
            var_wrf_2[i,:,:]  = nc_wrf.variables['RAINC'][loop[i],:,:]  
            var_wrf_11[i,:,:] = nc_wrf.variables['RAINNC'][loop[i]-1,:,:]
            var_wrf_22[i,:,:] = nc_wrf.variables['RAINC'][loop[i]-1,:,:]  
            var_wrf_111[i,:,:] = (var_wrf_2[i,:,:]+var_wrf_1[i,:,:])-(var_wrf_22[i,:,:]+var_wrf_11[i,:,:])
            sleep(0.01) 
if variable.value == 3.2:
    for i in tqdm(range(len_loop), desc='Prec'):
        if i == 0:
            var_wrf_1[0,:,:] = nc_wrf.variables['RAINNC'][0,:,:]
            var_wrf[0,:,:]   = var_wrf_1[0,:,:]
            sleep(0.01) 
        elif i >= 1:
            var_wrf_1[i,:,:]  = nc_wrf.variables['RAINNC'][loop[i],:,:]
            var_wrf_2[i,:,:]  = nc_wrf.variables['RAINC'][loop[i],:,:]  
            var_wrf_11[i,:,:] = nc_wrf.variables['RAINNC'][loop[i]-1,:,:]
            var_wrf_22[i,:,:] = nc_wrf.variables['RAINC'][loop[i]-1,:,:]  

            var_wrf_111[i,:,:] = (var_wrf_2[i,:,:]+var_wrf_1[i,:,:])-(var_wrf_22[i,:,:]+var_wrf_11[i,:,:])
            var_wrf[i,:,:]     = var_wrf_111[i,:,:]+var_wrf[i-1,:,:]
 
    del var_wrf_111,var_wrf_22,var_wrf_11,var_wrf_2,var_wrf_1
if variable.value == 4:
    for i in tqdm(range(len_loop), desc='Heat Flux'):
        var_wrf_1[i,:,:] = nc_wrf.variables['LH'][loop[i],:,:]
        var_wrf_2[i,:,:] = nc_wrf.variables['HFX'][loop[i],:,:] 
        sleep(0.01) 
    var_wrf = var_wrf_1+var_wrf_2
if variable.value == 5:
    for i in tqdm(range(len_loop), desc='CAPE'):
        var_wrf[:,i,:,:] = getvar(nc_wrf, "cape_2d", loop[i])  
        sleep(0.01) 
if variable.value == 6:
    for i in tqdm(range(len_loop), desc='OLR'):
        var_wrf[i,:,:] = nc_wrf.variables['OLR'][loop[i],:,:]
        sleep(0.01) 

#### 5. Adjust colorbar ranges, plot map and then save the file. 

In [9]:
for i in tqdm(range(len_loop), desc='Maps'):
    m = Basemap(projection='merc',llcrnrlat=bbox[2],urcrnrlat=bbox[3],llcrnrlon=bbox[0],urcrnrlon=bbox[1], lat_ts=30,resolution='i')
    fig  = plt.figure(1,figsize=(10,8))  
    plt.xlabel('Longitude'u' [\N{DEGREE SIGN}]',labelpad=18,size=9)
    plt.ylabel('Latitude'u' [\N{DEGREE SIGN}]',labelpad=33,size=9)
    plt.title(timestr[i], fontsize=9)
    ax   = fig.add_subplot(111) 
    m.drawparallels(np.arange(-90.,120.,1.), linewidth=0.00, color='black', labels=[1,0,0,1],labelstyle="N/S",fontsize=10)
    m.drawmeridians(np.arange(-180.,180.,2.), linewidth=0.00,color='black', labels=[1,0,0,1],labelstyle="N/S",fontsize=10)

    #m.fillcontinents(color = '#bababa')
    m.drawcountries(color = '#000000',linewidth=0.5)
    m.drawcoastlines(color = '#000000',linewidth=0.5)
    m.drawstates(color = '#000000',linewidth=0.5)  

    if plot_currents.value==True:
        x_rho, y_rho = m(lon,lat)
        nsub  = 7
        scale = 0.065
        C = ax.quiver(x_rho[::nsub,::nsub],y_rho[::nsub,::nsub],u[i,::nsub,::nsub],v[i,::nsub,::nsub],alpha=0.5,scale=1/scale, zorder=1e35, width=0.0025,color='black',pivot='middle')
        qk = ax.quiverkey(C, .14, -0.130, 0.5, ' Sea Surface Current\n 0.5 m.s⁻¹ ', coordinates='axes',color='#444444',labelsep=0.05, labelcolor='black',alpha=1,fontproperties={'size': '9'})
    
    if plot_wind.value==True:
        x, y    = m(to_np(lons_wrf2), to_np(lats_wrf2))
        spacing = 80
        scale   = 0.05
        W = ax.quiver(x[::spacing,::spacing], y[::spacing,::spacing], to_np(uvmet10[0,i,::spacing, ::spacing]),to_np(uvmet10[1,i,::spacing, ::spacing]),pivot='middle',scale=8/scale, zorder=1e35, width=0.010,color='black',headlength=3, headaxislength=2.8,alpha=0.5)
        wk = ax.quiverkey(W, 0.83, -0.13, 10, 'Wind Vector at 10 m\n 10 m.s⁻¹ ', coordinates='axes',color='black',labelsep=0.05,labelcolor='black',fontproperties={'size': '9'})

    if plot_slp.value==True:
        clevs_slp   = np.arange(900,1030,1)
        x_slp, y_slp = m(to_np(lons_wrf), to_np(lats_wrf))
        Ct_slp       = ax.contour(gaussian_filter(x_slp,10),gaussian_filter(y_slp,10),gaussian_filter(to_np(slp[i,:,:]),10),clevs_slp,colors='blue',latlon=True,linewidths=1,linestyles='solid')
        clbls_slp    = plt.clabel(Ct_slp,fmt='%i',inline=1, fontsize=9)

    if plot_bathy.value==True:
        #bathy_levels     = [-1000,-200,-100,-50]
        bathy_levels     = [-1000,-200]        
        bathy_lon,bathy_lat,bathy_topo = create_bathy(lon_1=bbox[0],lon_2=bbox[1],lat_1=bbox[2],lat_2=bbox[3])
        Ct               = m.contour(gaussian_filter(bathy_lon,2),gaussian_filter(bathy_lat,2),gaussian_filter(bathy_topo,2),bathy_levels,colors='black',latlon=True,linewidths=0.7,linestyles='solid')
        #manual_locations = [(273937,29695),(298089,23657),(368129,16411),(409187,20034)]
        manual_locations = [(368129,16411),(409187,20034)]       
        clbls            = plt.clabel(Ct,fmt='%i', fontsize=9,manual=manual_locations,colors="black")

    if variable.value == 1:
        clevs = np.arange(16,26.1,0.1)
        ticks = np.arange(min(clevs),max(clevs),1) 
        cmap  = cmocean.cm.thermal  
        h1    = m.contourf(lon_var, lat_var, var_roms[i,:,:], clevs,latlon=True,cmap=cmap,extend="both")  
        cax   = fig.add_axes([0.37, 0.038, 0.27, 0.025]) 
        cb    = fig.colorbar(h1, cax=cax, orientation="horizontal",panchor=(0.5,0.5),shrink=0.3,ticks=ticks)   
        cb.set_label(r'Sea Surface Temperature [$^\circ\!$C]', fontsize=9, color='0.2',labelpad=0)
        cb.ax.tick_params(labelsize=9, length=2, color='0.2', labelcolor='0.2',direction='in') 
        cb.set_ticks(ticks)
    elif variable.value  == 2:
        clevs = np.arange(16,25.1,0.01)
        ticks = np.arange(min(clevs),max(clevs),1)  
        cmap  = cmocean.cm.haline  
        h1    = m.contourf(lon_var, lat_var, var_roms[i,:,:], clevs,latlon=True,cmap=cmap,extend="both")  
        cax   = fig.add_axes([0.37, 0.038, 0.27, 0.025]) 
        cb    = fig.colorbar(h1, cax=cax, orientation="horizontal",panchor=(0.5,0.5),shrink=0.3,ticks=ticks)   
        cb.set_label(r'Sea Surface Salinity [PSU]', fontsize=9, color='0.2',labelpad=-0)
        cb.ax.tick_params(labelsize=9, length=2, color='0.2', labelcolor='0.2',direction='in') 
        cb.set_ticks(ticks)
    elif variable.value == 3.1:
        clevs  = np.arange(0,53,1)
        ticks = np.arange(min(clevs),max(clevs),10)  
        cmap   = cmocean.cm.rain 
        x1, y1 = m(to_np(lons_wrf2), to_np(lats_wrf2))
        h1     = ax.contourf(x1, y1, to_np(var_wrf[i,:,:]), clevs,cmap=cmap,latlon=True,extend="both")    
        cax    = fig.add_axes([0.37, 0.038, 0.27, 0.025]) 
        cb    = fig.colorbar(h1, cax=cax, orientation="horizontal",panchor=(0.5,0.5),shrink=0.3,ticks=ticks)   
        cb.set_label(r'Daily Precipitation [mm]', fontsize=9, color='0.2',labelpad=0)
        cb.ax.tick_params(labelsize=9, length=2, color='0.2', labelcolor='0.2',direction='in') 
        cb.set_ticks(ticks)
    elif variable.value == 3.2:
        clevs  = np.arange(0,203,1)
        ticks = np.arange(min(clevs),max(clevs),50)  
        cmap   = cmocean.cm.rain 
        x1, y1 = m(to_np(lons_wrf2), to_np(lats_wrf2))
        h1     = ax.contourf(x1, y1, to_np(var_wrf[i,:,:]), clevs,cmap=cmap,latlon=True,extend="both")    
        cax    = fig.add_axes([0.37, 0.038, 0.27, 0.025]) 
        cb    = fig.colorbar(h1, cax=cax, orientation="horizontal",panchor=(0.5,0.5),shrink=0.3,ticks=ticks)   
        cb.set_label(r'Accumulated Precipitation [mm]', fontsize=9, color='0.2',labelpad=0)
        cb.ax.tick_params(labelsize=9, length=2, color='0.2', labelcolor='0.2',direction='in') 
        cb.set_ticks(ticks)

    elif variable.value == 4:
        clevs  = np.arange(0,415,5)
        ticks = np.arange(min(clevs),max(clevs),100)  
        cmap   = cmocean.cm.thermal  
        x1, y1 = m(to_np(lons_wrf2), to_np(lats_wrf2))
        h1     = ax.contourf(x1, y1, var[i,:,:], clevs,cmap=cmap,latlon=True,extend="both")   
        cax    = fig.add_axes([0.37, 0.038, 0.27, 0.025]) 
        cb     = fig.colorbar(h1, cax=cax, orientation="horizontal",panchor=(0.5,0.5),shrink=0.3,ticks=ticks)   
        cb.set_label(r'Heat Fluxes [W m.s⁻²]', fontsize=9, color='0.2',labelpad=0) 
        cb.ax.tick_params(labelsize=9, length=2, color='0.2', labelcolor='0.2',direction='in') 
        cb.set_ticks(ticks)        
    elif variable.value == 5:
        clevs  = np.arange(0,820,10)
        ticks  = np.arange(min(clevs),max(clevs),200)  
        cmap   = plt.jet() #cmocean.cm.thermal  
        x1, y1 = m(to_np(lons_wrf2), to_np(lats_wrf2))
        h1     = ax.contourf(x1, y1, var_wrf[0,i,:,:], clevs,cmap=cmap,latlon=True,extend="both")   
        cax    = fig.add_axes([0.37, 0.038, 0.27, 0.025]) 
        cb     = fig.colorbar(h1, cax=cax, orientation="horizontal",panchor=(0.5,0.5),shrink=0.3,ticks=ticks)   
        cb.set_label(r'Maximum Convective Available Potential Energy [J kg⁻¹]', fontsize=9, color='0.2',labelpad=0) 
        cb.ax.tick_params(labelsize=9, length=2, color='0.2', labelcolor='0.2',direction='in') 
        cb.set_ticks(ticks)      
    elif variable.value == 6:
        clevs  = np.arange(0,400,5)
        ticks  = np.arange(min(clevs),max(clevs),10)  
        cmap   = cmocean.cm.gray_r
        x1, y1 = m(to_np(lons_wrf2), to_np(lats_wrf2))
        h1     = ax.contourf(x1, y1, var_wrf[i,:,:], clevs,cmap=cmap,latlon=True,extend="both")   
        cax    = fig.add_axes([0.37, 0.038, 0.27, 0.025]) 
        cb     = fig.colorbar(h1, cax=cax, orientation="horizontal",panchor=(0.5,0.5),shrink=0.3,ticks=ticks)   
        cb.set_label(r'Outgoing Longwave Radiation [W m.s⁻²]', fontsize=9, color='0.2',labelpad=0) 
        cb.ax.tick_params(labelsize=9, length=2, color='0.2', labelcolor='0.2',direction='in') 
        cb.set_ticks(ticks) 

    # Save figures.
    if variable.value == 1:
        try:
            os.makedirs("sst")
        except FileExistsError:
            pass
        if ppt_fig.value==True:
            plt.savefig('./sst/temp_{0:03d}.png'.format(i), transparent=True, bbox_inches = 'tight', pad_inches=0, dpi=100)
        elif ppt_fig.value==False:
            plt.savefig('./sst/temp_{0:03d}.png'.format(i), transparent=False, bbox_inches = 'tight', pad_inches=0, dpi=100)        
        plt.clf()   
    elif variable.value == 2:
        try:
            os.makedirs("sss")
        except FileExistsError:
            pass
        if ppt_fig.value==True:
            plt.savefig('./sss/salt_{0:03d}.png'.format(i), transparent=True, bbox_inches = 'tight', pad_inches=0, dpi=100)
        elif ppt_fig.value==False:
            plt.savefig('./sss/salt_{0:03d}.png'.format(i), transparent=False, bbox_inches = 'tight', pad_inches=0, dpi=100)            
        plt.clf()   
    elif variable.value == 3.1:
        try:
            os.makedirs("daily_precipitation")
        except FileExistsError:
            pass
        if ppt_fig.value==True:
            plt.savefig('./daily_precipitation/prec_{0:03d}.png'.format(i), transparent=True, bbox_inches = 'tight', pad_inches=0, dpi=100)
        elif ppt_fig.value==False:
            plt.savefig('./daily_precipitation/prec_{0:03d}.png'.format(i), transparent=False, bbox_inches = 'tight', pad_inches=0, dpi=100)           
        plt.clf()
    elif variable.value == 3.2:
        try:
            os.makedirs("accum_precipitation")
        except FileExistsError:
            pass
        if ppt_fig.value==True:
            plt.savefig('./accum_precipitation/prec_{0:03d}.png'.format(i), transparent=True, bbox_inches = 'tight', pad_inches=0, dpi=100)
        elif ppt_fig.value==False:
            plt.savefig('./accum_precipitation/prec_{0:03d}.png'.format(i), transparent=False, bbox_inches = 'tight', pad_inches=0, dpi=100)           
        plt.clf()       
    elif variable.value == 4:
        try:
            os.makedirs("heat_fluxes")
        except FileExistsError:
            pass
        if ppt_fig.value==True:
            plt.savefig('./heat_fluxes/fluxes_{0:03d}.png'.format(i), transparent=True, bbox_inches = 'tight', pad_inches=0, dpi=100)
        elif ppt_fig.value==False:
            plt.savefig('./heat_fluxes/fluxes_{0:03d}.png'.format(i), transparent=False, bbox_inches = 'tight', pad_inches=0, dpi=100)            
        plt.clf()
    elif variable.value== 5:
        try:
            os.makedirs("cape")
        except FileExistsError:
            pass
        if variable.value==True:
            plt.savefig('./cape/cape_{0:03d}.png'.format(i), transparent=True, bbox_inches = 'tight', pad_inches=0, dpi=100)
        elif ppt_fig.value==False:
            plt.savefig('./cape/cape_{0:03d}.png'.format(i), transparent=False, bbox_inches = 'tight', pad_inches=0, dpi=100)          
        plt.clf()   
    elif variable.value== 6:
        try:
            os.makedirs("olr")
        except FileExistsError:
            pass
        if variable.value==True:
            plt.savefig('./olr/olr_{0:03d}.png'.format(i), transparent=True, bbox_inches = 'tight', pad_inches=0, dpi=100)
        elif ppt_fig.value==False:
            plt.savefig('./olr/olr_{0:03d}.png'.format(i), transparent=False, bbox_inches = 'tight', pad_inches=0, dpi=100)          
        plt.clf()   

# 4. Create mp4 file from figures.
if create_video.value==True:
    cwd = os.getcwd()
    if variable.value == 1:
        exists = os.path.isfile('./sst/wrf_roms_temp.mp4')
        if exists==True:
            os.system("rm -rf ./sst/wrf_roms_temp.mp4")
            os.system("ffmpeg -r 10 -pattern_type glob -i '"+cwd+"/sst/*.png' -c:v libx264 -pix_fmt yuv420p -crf 15 ./sst/wrf_roms_temp.mp4")
        else:
            os.system("ffmpeg -r 10 -pattern_type glob -i '"+cwd+"/sst/*.png' -c:v libx264 -pix_fmt yuv420p -crf 15 ./sst/wrf_roms_temp.mp4")
    if variable.value== 2:
        exists = os.path.isfile('./sss/wrf_roms_salt.mp4')
        if exists==True:
            os.system("rm -rf ./salinity/wrf_roms_salt.mp4")
            os.system("ffmpeg -r 10 -pattern_type glob -i '"+cwd+"/sss/*.png' -c:v libx264 -pix_fmt yuv420p -crf 15 ./sss/wrf_roms_salt.mp4")
        else:
            os.system("ffmpeg -r 10 -pattern_type glob -i '"+cwd+"/sss/*.png' -c:v libx264 -pix_fmt yuv420p -crf 15 ./sss/wrf_roms_salt.mp4")
    if variable.value== 3.1:
        exists = os.path.isfile('./daily_precipitation/wrf_roms_prec.mp4')
        if exists==True:
            os.system("rm -rf ./daily_precipitation/wrf_roms_prec.mp4")
            os.system("ffmpeg -r 10 -pattern_type glob -i '"+cwd+"/daily_precipitation/*.png' -c:v libx264 -pix_fmt yuv420p -crf 15 ./daily_precipitation/wrf_roms_prec.mp4")
        else:
            os.system("ffmpeg -r 10 -pattern_type glob -i '"+cwd+"/daily_precipitation/*.png' -c:v libx264 -pix_fmt yuv420p -crf 15 ./daily_precipitation/wrf_roms_prec.mp4")
    if variable.value== 3.2:
        exists = os.path.isfile('./accum_precipitation/wrf_roms_prec.mp4')
        if exists==True:
            os.system("rm -rf ./accum_precipitation/wrf_roms_prec.mp4")
            os.system("ffmpeg -r 10 -pattern_type glob -i '"+cwd+"/accum_precipitation/*.png' -c:v libx264 -pix_fmt yuv420p -crf 15 ./accum_precipitation/wrf_roms_prec.mp4")
        else:
            os.system("ffmpeg -r 10 -pattern_type glob -i '"+cwd+"/accum_precipitation/*.png' -c:v libx264 -pix_fmt yuv420p -crf 15 ./accum_precipitation/wrf_roms_prec.mp4")          
    if variable.value== 4:
        exists = os.path.isfile('./heat_fluxes/wrf_roms_flux.mp4')
        if exists==True:
            os.system("rm -rf ./heat_fluxes/wrf_roms_flux.mp4")
            os.system("ffmpeg -r 10 -pattern_type glob -i '"+cwd+"/heat_fluxes/*.png' -c:v libx264 -pix_fmt yuv420p -crf 15 ./heat_fluxes/wrf_roms_flux.mp4")
        else:
            os.system("ffmpeg -r 10 -pattern_type glob -i '"+cwd+"/heat_fluxes/*.png' -c:v libx264 -pix_fmt yuv420p -crf 15 ./heat_fluxes/wrf_roms_flux.mp4")
    if variable.value == 5:
        exists = os.path.isfile('./cape/wrf_roms_cape.mp4')
        if exists==True:
            os.system("rm -rf ./cape/wrf_roms_cape.mp4")
            os.system("ffmpeg -r 10 -pattern_type glob -i '"+cwd+"/cape/*.png' -c:v libx264 -pix_fmt yuv420p -crf 15 ./cape/wrf_roms_cape.mp4")
        else:
            os.system("ffmpeg -r 10 -pattern_type glob -i '"+cwd+"/cape/*.png' -c:v libx264 -pix_fmt yuv420p -crf 15 ./cape/wrf_roms_cape.mp4")
    if variable.value== 6:
        exists = os.path.isfile('./olr/wrf_roms_flux.mp4')
        if exists==True:
            os.system("rm -rf ./olr/wrf_roms_flux.mp4")
            os.system("ffmpeg -r 10 -pattern_type glob -i '"+cwd+"/olr/*.png' -c:v libx264 -pix_fmt yuv420p -crf 15 ./olr/wrf_roms_flux.mp4")
        else:
            os.system("ffmpeg -r 10 -pattern_type glob -i '"+cwd+"/olr/*.png' -c:v libx264 -pix_fmt yuv420p -crf 15 ./olr/wrf_roms_flux.mp4")

            
